In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pycaret
from pycaret.classification import *
from pycaret.datasets import get_data

ModuleNotFoundError: No module named 'pycaret4'

In [8]:
import pandas as pd
from pycaret.regression import *

train_data = pd.read_csv('cleaned_data.csv')

train_data.drop(columns=['Unnamed: 0'], inplace=True)

train_data['DATETIMEDATA'] = pd.to_datetime(train_data['DATETIMEDATA'])

regression_setup = setup(train_data, target='PM25', session_id=123, date_features=['DATETIMEDATA'], normalize=True)
best = compare_models()

rf_model = create_model('rf')

tuned_rf_model = tune_model(rf_model)

bagged_model = ensemble_model(tuned_rf_model, n_estimators=20)

last_prediction_time = train_data['DATETIMEDATA'].max()

start_date = last_prediction_time + pd.DateOffset(hours=1)
end_date = start_date + pd.DateOffset(days=30, hours=23)
next_week_hours = pd.date_range(start=start_date, end=end_date, freq='D')

next_week_data = pd.DataFrame({'DATETIMEDATA': next_week_hours})

next_week_data['O3'] = 0
next_week_data['WS'] = 0
next_week_data['TEMP'] = 0
next_week_data['RH'] = 0
next_week_data['WD'] = 0

model_predictions = predict_model(bagged_model, data=next_week_data)

mpdf = pd.DataFrame(model_predictions, columns=['DATETIMEDATA', 'prediction_label'])

print(mpdf)

,Description,Value
0,Session id,123
1,Target,PM25
2,Target type,Regression
3,Original data shape,"(1680, 7)"
4,Transformed data shape,"(1680, 9)"
5,Transformed train set shape,"(1176, 9)"
6,Transformed test set shape,"(504, 9)"
7,Numeric features,5
8,Date features,1
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,3.3166,19.6030,4.3973,0.3374,0.2241,0.1950,0.0320
rf,Random Forest Regressor,3.3826,19.7987,4.4327,0.3275,0.2290,0.2019,0.0410
gbr,Gradient Boosting Regressor,3.4791,20.3253,4.4890,0.3135,0.2360,0.2102,0.0180
lightgbm,Light Gradient Boosting Machine,3.4498,20.6801,4.5312,0.2977,0.2341,0.2045,0.0700
knn,K Neighbors Regressor,3.6882,23.2569,4.7944,0.2134,0.2501,0.2196,0.0120
ada,AdaBoost Regressor,3.8890,23.9462,4.8763,0.1915,0.2627,0.2463,0.0160
br,Bayesian Ridge,3.9656,25.8958,5.0671,0.1290,0.2699,0.2450,0.0070
ridge,Ridge Regression,3.9635,25.8894,5.0668,0.1287,0.2696,0.2446,0.0080
lar,Least Angle Regression,3.9635,25.8899,5.0669,0.1287,0.2696,0.2446,0.0070
lr,Linear Regression,3.9635,25.8899,5.0669,0.1287,0.2696,0.2446,0.2990


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.0089,14.8215,3.8499,0.4163,0.2195,0.1956
1,3.4838,20.7948,4.5601,0.3118,0.2193,0.1944
2,3.2414,16.9141,4.1127,0.3317,0.2246,0.2017
3,3.5509,24.2628,4.9257,0.2259,0.2458,0.2120
4,3.5883,21.2678,4.6117,0.2562,0.2402,0.2131
5,3.6544,25.9926,5.0983,0.3756,0.2414,0.2040
6,3.1608,15.6374,3.9544,0.3911,0.2167,0.1925
7,3.1236,17.5463,4.1888,0.3990,0.2087,0.1824
8,3.3860,19.5075,4.4167,0.1054,0.2304,0.1975


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.0990,15.6924,3.9614,0.3821,0.2276,0.2023
1,3.4968,19.2848,4.3914,0.3618,0.2147,0.1946
2,3.1669,16.1535,4.0191,0.3618,0.2202,0.1987
3,3.7309,25.3306,5.0329,0.1919,0.2550,0.2261
4,3.5645,20.8732,4.5687,0.2700,0.2386,0.2121
5,3.6407,26.1276,5.1115,0.3723,0.2417,0.2010
6,3.2910,16.5311,4.0658,0.3563,0.2285,0.2086
7,3.1222,17.9191,4.2331,0.3862,0.2134,0.1854
8,3.2978,18.3336,4.2818,0.1593,0.2302,0.1969


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.9310,14.0705,3.7511,0.4459,0.2178,0.1923
1,3.5284,19.9998,4.4721,0.3381,0.2166,0.1962
2,3.2396,16.4051,4.0503,0.3518,0.2215,0.2018
3,3.5593,23.6181,4.8598,0.2465,0.2437,0.2137
4,3.5242,20.8732,4.5687,0.2700,0.2374,0.2106
5,3.6583,25.9266,5.0918,0.3772,0.2415,0.2032
6,3.1818,15.6295,3.9534,0.3914,0.2182,0.1971
7,3.1272,17.3558,4.1660,0.4055,0.2109,0.1857
8,3.2711,18.8325,4.3396,0.1364,0.2293,0.1945


   DATETIMEDATA  prediction_label
0    2024-03-11         22.111816
1    2024-03-12         22.204232
2    2024-03-13         21.801291
3    2024-03-14         21.682245
4    2024-03-15         21.016929
5    2024-03-16         20.768957
6    2024-03-17         20.456443
7    2024-03-18         20.258216
8    2024-03-19         20.658669
9    2024-03-20         21.419819
10   2024-03-21         21.587177
11   2024-03-22         21.139065
12   2024-03-23         19.099157
13   2024-03-24         18.951667
14   2024-03-25         18.531671
15   2024-03-26         18.739036
16   2024-03-27         19.149325
17   2024-03-28         19.422577
18   2024-03-29         19.506243
19   2024-03-30         19.491853
20   2024-03-31         19.420513
21   2024-04-01         22.527963
22   2024-04-02         22.519143
23   2024-04-03         21.862883
24   2024-04-04         21.755337
25   2024-04-05         23.005016
26   2024-04-06         23.307866
27   2024-04-07         23.939709
28   2024-04-0

In [9]:
mpdf.to_csv('model_predictions.csv', index=False)